In [615]:
import numpy as np
import scipy 
import seaborn as sns
import pandas as pd
import patsy
import matplotlib
import matplotlib.pyplot as plt
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
import requests
import random
import sys
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
plt.style.use('fivethirtyeight')


In [536]:
#                                      importing list of user numbers
# users1 = pd.read_csv('/Users/caitlinmowdy/users2500.csv')
# users2 = pd.read_csv('/Users/caitlinmowdy/users2.csv')
# users3 = pd.read_csv('/Users/caitlinmowdy/user3.csv')

# user_id_list=[]
# for x in users1['0'].unique():
#     user_id_list.append(x)
# for x in users2['0'].unique():
#     user_id_list.append(x)
# for x in users3['0'].unique():
#     user_id_list.append(x)


In [574]:
# import user_ids
real_users1 = pd.read_csv('/Users/caitlinmowdy/real_users1.csv')
real_users2 = pd.read_csv('/Users/caitlinmowdy/real_users2.csv')

In [584]:
# put user_ids in a lits
real_user_id_list=[]
for x in real_users1['0'].unique():
    real_user_id_list.append(x)
for x in real_users2['0'].unique():
    real_user_id_list.append(x)

In [640]:
len(real_user_id_list)

2827

In [599]:
# init DataFrames 

revs_df = pd.DataFrame()
users_df = pd.DataFrame()
hostels_df = pd.DataFrame()


In [638]:
# Get details for user and rev DFs

for i,user in enumerate(real_user_id_list[2382:]):
    
    rv_df = pd.DataFrame()
    us_df = pd.DataFrame()
    X = 'http://www.hostelworld.com/profile/' + str(user) + '/reviews'
    response = requests.get(X)
    HTML = response.text
    
    rv_df['text'] = Selector(text=HTML).xpath('//div[@id]/p/text()').extract()
    rv_df['user_id'] = [user]*len(rv_df['text'])
    rv_df['score'] = Selector(text=HTML).xpath("//div[@class='textrating']/text()").extract()
    rv_df['hostel'] = Selector(text=HTML).xpath("//div[@class='popupreviewlocation']/a/text()").extract()
    rv_df['location'] = Selector(text=HTML).xpath("//div[@class='popupreviewlocation']/span/text()").extract()
    rv_df['link'] = Selector(text=HTML).xpath("//div[@class='popupreviewlocation']/a/@href").extract()
    
    us_df['user_id'] = [user]
    us_df['about'] = [Selector(text=HTML).xpath("//li[@class='reviewerdetails']/text()").extract()[0]] 
    us_df['num_revs']=len(rv_df['text'])
    # us_df['avg_score'] = np.mean(df1['score']) 
     
    if i%10 == 0:
        sys.stdout.write('\r'+ str((float(i)/len(real_user_id_list))*100)+'%')
     
    users_df = users_df.append(us_df,ignore_index=True)
    revs_df = revs_df.append(rv_df,ignore_index=True)
    

15.5642023346%

In [639]:
users_df.shape, revs_df.shape

((2828, 3), (5809, 6))

In [ ]:
# get details for hostel DF
hostels_df = pd.DataFrame()
for i,url in enumerate(revs_df.link):
    hst_df = pd.DataFrame()
    X = str(url)
    response = requests.get(X)
    HTML = response.text
    
    free = Selector(text=HTML).xpath('//li[1]/div/p/text()').extract()
    general = Selector(text=HTML).xpath('//li[2]/div/p').extract()
    services = Selector(text=HTML).xpath('//li[3]/div/p').extract()
    food_drink = Selector(text=HTML).xpath('//li[4]/div/p').extract()
    entertainment = Selector(text=HTML).xpath('//li[5]/div/p').extract()
    policies = Selector(text=HTML).xpath("//div[@id='panel4']/p").extract()
    ratings = Selector(text=HTML).xpath("//div[@class='ratingbox']/text()").extract()
    description = Selector(text=HTML).xpath("//div[@class='section-propertytext']/div[@class='prop-intro']/div[@class='microdetailstext hiddentext prop-text bigtext ']/text()").extract()
    hst = revs_df.hostel.iloc[i]
    
    if i%10 == 0:
        sys.stdout.write('\r'+ str((float(i)/len(revs_df.link))*100)+'%')
        
    if len(ratings) != 0:
        if float(ratings[0]) != 0.0:

            hst_df['rating'] = [float(ratings[0])]
            hst_df['desc'] = [description]
            hst_df['FREE'] = [free]
            hst_df['GENERAL'] = [general] 
            hst_df['SERVICES'] = [services]
            hst_df['FOOD_DRINK'] = [food_drink]
            hst_df['ENTERTAINMENT'] = [entertainment]
            hst_df['POLICIES'] = [policies]
            hst_df['hostel']= hst
        
            hostels_df = hostels_df.append(hst_df,ignore_index=True)
            
        # else statment to del a rev from revs_df is the hostel no longer exists
        else:
            revs_df.drop(revs_df.index[i], inplace=True)
    # else statment to del a rev from revs_df is the hostel no longer exists
    else:
        revs_df.drop(revs_df.index[i], inplace=True)

17.895185343%

In [691]:
hostels_df.head()

,rating,desc,FREE,GENERAL,SERVICES,FOOD_DRINK,ENTERTAINMENT,POLICIES,hostel
0,8.4,[\n Please confirm your che...,[\n ...,[<p>\n ...,[<p>\n ...,[<p>\n ...,[],"[<p>\n No Curfew<i class=""fa fa-cir...",Hostel Bell
1,8.5,[\n Le Beau Site is a welco...,[\n ...,[<p>\n ...,[<p>\n ...,[<p>\n ...,[],"[<p>\n Child Friendly<i class=""fa f...",Beau Site
2,8.2,[\n Old Prague Hostel is on...,[\n ...,[<p>\n ...,[<p>\n ...,[<p>\n ...,[<p>\n ...,"[<p>\n No Curfew<i class=""fa fa-cir...",Old Prague Hostel
3,8.2,[\n Â« Special offer till 2...,[\n ...,[<p>\n ...,[<p>\n ...,[<p>\n ...,[<p>\n ...,[<p>\n Credit Cards Accepted<i clas...,Le Montclair Montmartre Hostel & Budget Hotel
4,8.0,[\n Charming hotel two star...,[\n ...,[<p>\n ...,[<p>\n ...,[],[],[<p>\n Credit Cards Accepted<i clas...,Hotel de la Tour Eiffel


In [ ]:
# code to remove html tags from data 
import re
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [ ]:
#   df1['age'] = Selector(text=HTML).xpath("//li[@class='reviewerrank']/text()").extract()
#   df1['nationality'] = Selector(text=HTML).xpath("//li[@class='reviewerdetails']/text()").extract()

# df["country"],df["group"],df["age"] = zip(*df["about"].str.split().tolist())
# del df['about']
# df.head()

In [ ]:
#         new_hostels['GENERAL'] = Selector(text=HTML).xpath('//li[2]/div/p').extract()
#         new_hostels['SERVICES'] = Selector(text=HTML).xpath('//li[3]/div/p').extract()
#         new_hostels['FOOD_DRINK'] = Selector(text=HTML).xpath('//li[4]/div/p').extract()
#         new_hostels['ENTERTAINMENT'] = Selector(text=HTML).xpath('//li[5]/div/p').extract()
#         new_hostels['POLICIES'] = Selector(text=HTML).xpath("//div[@id='panel4']/p").extract()

In [ ]:
#                                           test with one user number

# df_test = pd.DataFrame()

# X = 'http://www.hostelworld.com/profile/14645677/reviews'
# response = requests.get(X)
# HTML = response.text

# df_test['user'] =[14645677]*4
# df_test['text'] = Selector(text=HTML).xpath('//div[@id]/p/text()').extract()
# df_test['score'] = Selector(text=HTML).xpath("//div[@class='textrating']/text()").extract()
# df_test['hostel'] = Selector(text=HTML).xpath("//div[@class='popupreviewlocation']/a/text()").extract()
# df_test['location'] = Selector(text=HTML).xpath("//div[@class='popupreviewlocation']/span/text()").extract()
# df_test['about'] =  Selector(text=HTML).xpath("//li[@class='reviewerdetails']/text()").extract()

In [ ]:
# Split user.about into thee columns
# df_test["country"],df_test["group"],df_test["age"] = zip(*df_test["about"].str.split().tolist())
# del df_test['about']

In [ ]:
#                                          Hostel details for one hostel

# hostels = pd.DataFrame()
# X = 'http://www.hostelworld.com/hosteldetails.php/Fat-Cod-Backpackers/Picton/2899'
# response = requests.get(X)
# HTML = response.text

# hostels['rating'] = Selector(text=HTML).xpath("normalize-space(//div[@class='microsite_rating clearfix']/div[@class='ratingbox'])").extract()
# hostels['Description'] = [Selector(text=HTML).xpath("//div[@class='section-propertytext']/div[@class='prop-intro']/div[@class='microdetailstext hiddentext prop-text bigtext ']/text()").extract()]
# hostels['FREE'] = Selector(text=HTML).xpath('//li[1]/div/p').extract()
# hostels['GENERAL'] = Selector(text=HTML).xpath('//li[2]/div/p').extract()
# hostels['SERVICES'] = Selector(text=HTML).xpath('//li[3]/div/p').extract()
# hostels['FOOD_DRINK'] = Selector(text=HTML).xpath('//li[4]/div/p').extract()
# hostels['ENTERTAINMENT'] = Selector(text=HTML).xpath('//li[5]/div/p').extract()
# hostels['POLICIES'] = Selector(text=HTML).xpath("//div[@id='panel4']/p").extract()